# MNIST CNN

## How CNN work 

### Convolutional layer contain following Layers

#### convolutional
#### Relu layer
#### Pooling
#### Fully Connected

Full Detail edureka 3.53

In [1]:
import tensorflow as tf

In [2]:
import warnings

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
warnings.filterwarnings('ignore')
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Helper

### Init Weight

In [5]:
def init_weights(shape):
    init_random_dist=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

### Init Bias

In [6]:
def init_bias(shape):
    init_bias_vals=tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

### CONV 2D

convenience function

In [7]:
def conv2d(x,w):
    #x-->[batch,H,W,Channels]
    #w-->[filter H,filter W,channels IN,channel Out]
    
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

### Pooling

In [8]:
def max_pool_2by2(x):
    # x--> [batch,h,w,channels]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

### Convolutional layer

In [9]:
def convolutional_layer(input_x,shape):
    w=init_weights(shape)
    b=init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,w)+b)
    

### Normal(Fully Connected)

In [10]:
def normal_full_layer(input_layer,size):
    input_size=int(input_layer.get_shape()[1])
    w=init_weights([input_size,size])
    b=init_bias([size])
    return tf.matmul(input_layer,w)+b

### Placeholder

In [11]:
x=tf.placeholder(tf.float32,shape=[None,784])

In [12]:
y_true=tf.placeholder(tf.float32,shape=[None,10])

### Layers

In [13]:
x_image=tf.reshape(x,[-1,28,28,1])

In [14]:
convo_1=convolutional_layer(x_image,shape=[5,5,1,32])
convo_1_pooling=max_pool_2by2(convo_1)

In [15]:
convo_2=convolutional_layer(convo_1_pooling,shape=[5,5,32,64])
convo_2_pooling=max_pool_2by2(convo_2)

In [16]:
convo_2_flat=tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one=tf.nn.relu(normal_full_layer(convo_2_flat,1024))

### Dropout

In [17]:
hold_prob=tf.placeholder(tf.float32)
full_one_dropout=tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
y_pred=normal_full_layer(full_one_dropout,10)

### Loss Function

In [19]:
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Optimizer

In [20]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.001)
train=optimizer.minimize(cross_entropy)

In [21]:
init=tf.global_variables_initializer()

In [ ]:
steps=5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x,batch_y = mnist.train.next_batch(50)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        if i%100 == 0:
            
            print('ON STEP : {}'.format(i))
            print('ACCURACY:')
            
            matches=tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            
            acc=tf.reduce_mean(tf.cast(matches,tf.float32))
            
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

ON STEP : 0
ACCURACY:
